In [ ]:
from itertools import islice
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os
from tqdm import tqdm 
import sys
from datetime import datetime, timedelta, time
from scipy.optimize import curve_fit

# カレントディレクトリを.pyと合わせるために以下を実行
from pathlib import Path
if Path.cwd().name == "notebook":
    os.chdir("..")

# Windows MatplotlibのデフォルトフォントをMeiryoに設定
plt.rcParams['font.family'] = 'Meiryo'


# 設定
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
def replace_nan(df):
    df = df.replace('N', np.nan)
    df = df.replace('NaN', np.nan)
    df = df.replace('nan', np.nan)
    df = df.replace('foo', np.nan)
    df = df.replace('///', np.nan)
    return df

def set_dtype(df):
    column_types = {
        'id':np.float32,
        'user_id':np.float64,
        'series_id' : np.float32,
        'shop_id' : str,
        'shop_name' : str,
        'card_id' : str,
        'amount' : np.float32,
        'amount_kg' : np.float32,
        'point' : np.float32,
        'total_point' : np.float32,
        'total_amount' : np.float32,
        'coin' : np.float32,
        'rank_id':np.float32,
        'use_date': 'datetime64[ns]',
        'created_at': 'datetime64[ns]',
        'updated_at': 'datetime64[ns]',
        '支店ID' : np.float32,
        'super' : str,
        'prefectures' : str,
        'municipality' : str,
        'shop_name_1' :  str,
        'shop_id_1' :    str,
        'store_latitude' : np.double,
        'store_longitude' : np.double,
        '年月日' : 'datetime64[ns]',
        '天気': str,
        '平均気温(℃)': np.float32,
        '最高気温(℃)': np.float32,
        '最低気温(℃)': np.float32,
        '降水量の合計(mm)': np.float32,
        '平均風速(m/s)': np.float32,
        '平均湿度(％)': np.float32,
        '平均現地気圧(hPa)': np.float32,
        '平均雲量(10分比)': np.float32,
        '降雪量合計(cm)': np.float32,
        '日照時間(時間)': np.float32,
        '合計全天日射量(MJ/㎡)': np.float32,
    }
    df = df.astype(column_types)
    return df

def aggregate_shop_date(df):
    # shop_idと年月日ごとにグループ化し、合計値と代表値を計算
    aggregated_df = df.groupby(['shop_id', '年月日']).agg({
        'amount': 'sum',
        'amount_kg': 'sum',
        'point': 'sum',
        'total_point': 'sum',
        'total_amount': 'sum',
        'coin': 'sum',
        'series_id': 'first',
        'shop_name': 'first',
        'リサイクル分類ID': 'first',
        '支店ID': 'first',
        'super': 'first',
        'prefectures': 'first',
        'municipality': 'first',
        'shop_name_1': 'first',
        'shop_id_1': 'first',
        'store_opening_time': 'first',
        'store_closing_time': 'first',
        'rps_opening_time': 'first',
        'rps_closing_time': 'first',
        'store_latitude': 'first',
        'store_longitude': 'first',
        '天気': 'first',
        '平均気温(℃)': 'first',
        '最高気温(℃)': 'first',
        '最低気温(℃)': 'first',
        '降水量の合計(mm)': 'first',
        '平均風速(m/s)': 'first',
        '平均湿度(％)': 'first',
        '平均現地気圧(hPa)': 'first',
        '平均雲量(10分比)': 'first',
        '降雪量合計(cm)': 'first',
        '日照時間(時間)': 'first',
        '合計全天日射量(MJ/㎡)': 'first'
    }).reset_index()

    # shop_idと年月日でソート
    aggregated_df = aggregated_df.sort_values(by=['shop_id', '年月日'])

    # 結果を保存
    aggregated_df.to_csv('data/input/point_history_per_shop_date.csv', index=False, encoding="utf-8")

def aggregate_date(df):
    # shop_idごとにグループ化し、合計値と代表値を計算
    aggregated_df = df.groupby(['shop_id']).agg({
        'amount': 'sum',
        'amount_kg': 'sum',
        'point': 'sum',
        'total_point': 'sum',
        'total_amount': 'sum',
        'coin': 'sum',
        'series_id': 'first',
        'shop_name': 'first',
        'リサイクル分類ID': 'first',
        '支店ID': 'first',
        'super': 'first',
        'prefectures': 'first',
        'municipality': 'first',
        'shop_name_1': 'first',
        'shop_id_1': 'first',
        'store_opening_time': 'first',
        'store_closing_time': 'first',
        'rps_opening_time': 'first',
        'rps_closing_time': 'first',
        'store_latitude': 'first',
        'store_longitude': 'first',
        '天気': 'first',
        '平均気温(℃)': 'first',
        '最高気温(℃)': 'first',
        '最低気温(℃)': 'first',
        '降水量の合計(mm)': 'first',
        '平均風速(m/s)': 'first',
        '平均湿度(％)': 'first',
        '平均現地気圧(hPa)': 'first',
        '平均雲量(10分比)': 'first',
        '降雪量合計(cm)': 'first',
        '日照時間(時間)': 'first',
        '合計全天日射量(MJ/㎡)': 'first'
    }).reset_index()

    # shop_idでソート
    aggregated_df = aggregated_df.sort_values(by=['shop_id'])

    # 結果を保存
    aggregated_df.to_csv('data/input/point_history_per_shop.csv', index=False, encoding="utf-8")

# カスタム関数を定義
def parse_date(date):
    try:
        return pd.to_datetime(date)
    except ValueError:
        try:
            return pd.to_datetime(date, format='%Y年%m月%d日')
        except ValueError:
            return pd.to_datetime(date, format='%Y/%m/%d')

def replace_nan(df):
    df = df.replace('N', np.nan)
    df = df.replace('NaN', np.nan)
    df = df.replace('nan', np.nan)
    df = df.replace('foo', np.nan)
    df = df.replace('///', np.nan)
    return df


def set_dtype(df):
    # 列名を直感的に変更
    df = df.rename(columns={'id_1': '支店ID'})
    df = df.rename(columns={'item_id': 'リサイクル分類ID'})

    column_types = {
        'id':np.float32,
        'user_id':np.float64,
        'series_id' : np.float32,
        'shop_id' : str,
        'shop_name' : str,
        'card_id' : str,
        'リサイクル分類ID' : str,
        'amount' : np.float32,
        'amount_kg' : np.float32,
        'point' : np.float32,
        'total_point' : np.float32,
        'total_amount' : np.float32,
        'coin' : np.float32,
        'rank_id':np.float32,
        'use_date': 'datetime64[ns]',
        'created_at': 'datetime64[ns]',
        'updated_at': 'datetime64[ns]',
        '支店ID' : np.float32,
        'super' : str,
        'prefectures' : str,
        'municipality' : str,
        'shop_name_1' :  str,
        'shop_id_1' :    str,
        'store_latitude' : np.double,
        'store_longitude' : np.double,
        #'年月日' : 'datetime64[ns]',
        #'天気': str,
        #'平均気温(℃)': np.float32,
        #'最高気温(℃)': np.float32,
        #'最低気温(℃)': np.float32,
        #'降水量の合計(mm)': np.float32,
        #'平均風速(m/s)': np.float32,
        #'平均湿度(％)': np.float32,
        #'平均現地気圧(hPa)': np.float32,
        #'平均雲量(10分比)': np.float32,
        #'降雪量合計(cm)': np.float32,
        #'日照時間(時間)': np.float32,
        #'合計全天日射量(MJ/㎡)': np.float32,
    }
    df = df.astype(column_types)
    #df['rps_opening_time'] = pd.to_datetime(df['rps_opening_time']).dt.time
    #df['rps_closing_time'] = pd.to_datetime(df['rps_closing_time']).dt.time
    #df['store_opening_time'] = pd.to_datetime(df['store_opening_time']).dt.time
    #df['store_closing_time'] = pd.to_datetime(df['store_closing_time']).dt.time
    df['rps_opening_time'] = pd.to_datetime(df['use_date'].dt.date.astype(str) + ' ' + df['rps_opening_time'])
    df['rps_closing_time'] = pd.to_datetime(df['use_date'].dt.date.astype(str) + ' ' + df['rps_closing_time'])


    return df


In [ ]:

#concat_csv()
#df = pd.read_csv('data/input/point_history_ヨークベニマル_明石台店.csv', encoding="utf-8")
df = pd.read_csv('data/input/point_history_ヨークベニマル_南中山店.csv', encoding="utf-8")
df = replace_nan(df)
df = set_dtype(df)
# df[(pd.to_datetime(df['use_date']) < pd.to_datetime('2023-01-02')) & (pd.to_datetime(df['use_date']) > pd.to_datetime('2022-12-30'))].sort_values(by='use_date')


In [ ]:
# 年月日ごとにグループ化し、amount_kgの合計値をplot
df['年月日'] = pd.to_datetime(df['use_date']).dt.floor('d')
df.groupby('年月日')['amount_kg'].sum().plot()

In [ ]:
# use_date列の差分を計算
df['time_diff'] = df['use_date'].diff()

# df['年月日']について前の行と日付が異なる場合、df['rps_closing_time']とdf['rps_opening_time']の差をdf['time_diff']に格納
df.loc[df['年月日'].diff().dt.total_seconds() != 0, 'time_diff'] -= df['rps_closing_time'] - df['rps_opening_time']


df['time_diff'] = df['time_diff'].dt.total_seconds() / 3600

# 最初の行には nan を設定
df.loc[0, 'time_diff'] = np.nan

In [ ]:
# df['use_date'].diff().dt.days != 0 がtrueのインデックスを取得
df[df['use_date'].dt.date.diff().dt.days != 0].index

In [ ]:
fig, ax = plt.subplots()
ax.plot(df['use_date'],df['time_diff'])
#ax.scatter(df['use_date'],df['time_diff'], s=2)

# x軸のラベル表示間隔を調整
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

ax.set_xlabel('年月日')
ax.set_ylabel('リサイクルステーションの利用間隔[h]')

# yの最大値
ax.set_ylim(0, 6)



In [ ]:
# 期間を指定する変数の命名
start_date = '2023-03-21'
end_date = '2023-04-03'

# 指定した期間のデータを抽出
mask = (df['年月日'] >= start_date) & (df['年月日'] <= end_date)
filtered_df = df.loc[mask]

fig, ax = plt.subplots()
ax.scatter(filtered_df['use_date'],filtered_df['time_diff'], s=5)
#ax.plot(filtered_df['use_date'],filtered_df['time_diff'])
# x軸のラベル表示間隔を調整
ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

ax.set_xlabel('年月日')
ax.set_ylabel('リサイクルステーションの利用間隔[h]')

# yの最大値
#ax.set_ylim(0, 12)

In [ ]:
# df['use_date_diff']の分布を確認
fig, ax = plt.subplots()
ax.hist(df['time_diff'], bins=100, range=(0, 12))
ax.set_xlabel('リサイクルステーションの利用間隔[h]')
ax.set_ylabel('頻度')
# 両対数にする
ax.set_yscale('log')
ax.set_xscale('log')

グラフに描けば、両対数グラフにおいて、線型になる　→　冪乗則に従う
https://ja.wikipedia.org/wiki/%E5%86%AA%E4%B9%97%E5%89%87

In [ ]:

# ヒストグラムのデータを取得
counts, bin_edges = np.histogram(df['time_diff'], bins=100, range=(0, 12))

# ビンの中心を計算
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# ヒストグラムをプロット
fig, ax = plt.subplots()
ax.bar(bin_centers, counts, width=np.diff(bin_edges))
ax.set_xlabel('リサイクルステーションの利用間隔[h]')
ax.set_ylabel('頻度')
ax.set_yscale('log')

In [ ]:
# べき乗則関数を定義
def power_law(x, a, b):
    return a * np.power(x, b)

# ヒストグラムのデータを取得
counts, bin_edges = np.histogram(df['time_diff'], bins=100, range=(0, 12))
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# x軸の大きい値を重視してべき乗則のフィットを行う
mask = counts > 0
weights = 1 / bin_centers[mask] ** 2

# 重み付きフィットを実行
params, params_covariance = curve_fit(power_law, bin_centers[mask], counts[mask], sigma=weights)
print("params",params)

# フィット結果をプロット
fig, ax = plt.subplots()
ax.bar(bin_centers, counts, width=np.diff(bin_edges), label='Data')
ax.plot(bin_centers, power_law(bin_centers, *params), label='Fit: a=%.2f, b=%.2f' % tuple(params), color='red')
ax.set_xlabel('リサイクルステーションの利用間隔[h]')
ax.set_ylabel('Frequency')
ax.set_xscale('log')
ax.set_yscale('log')

In [ ]:
# 閾値を設定（x軸が0の部分は誤差が大きいため除外）
threshold = np.percentile(bin_centers[mask], 0)

# 閾値より大きい値のみを考慮
selected_mask = bin_centers[mask] > threshold

# 残差平方和（RSS）を計算
residuals = counts[mask][selected_mask] - power_law(bin_centers[mask][selected_mask], *params)
rss = np.sum(residuals**2)

# 全変動平方和（TSS）を計算
mean_counts = np.mean(counts[mask][selected_mask])
tss = np.sum((counts[mask][selected_mask] - mean_counts)**2)

# 決定係数（R²）を計算
r_squared = 1 - (rss / tss)

print(f"R-squared: {r_squared}")